# F1 2024 Database Setup
This notebook sets up the SQLite database for F1 2024 season data using the Ergast API.

## Database Structure
- **drivers**: Driver information
- **constructors**: Team/constructor information  
- **races**: Race calendar and details
- **results**: Race results
- **qualifying**: Qualifying results
- **driver_standings**: Championship standings for drivers
- **constructor_standings**: Championship standings for constructors

In [1]:
import sqlite3
import requests
import json
import pandas as pd
from datetime import datetime
import time
import os

# Create backend directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Database connection
DB_PATH = '../data/f1_2024.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

print("Database connection established")

Database connection established


In [2]:
# Create all tables
def create_tables():
    # Drivers table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS drivers (
        driver_id TEXT PRIMARY KEY,
        permanent_number INTEGER,
        code TEXT,
        given_name TEXT,
        family_name TEXT,
        date_of_birth TEXT,
        nationality TEXT,
        url TEXT
    )
    ''')
    
    # Constructors table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS constructors (
        constructor_id TEXT PRIMARY KEY,
        name TEXT,
        nationality TEXT,
        url TEXT
    )
    ''')
    
    # Races table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS races (
        race_id INTEGER PRIMARY KEY,
        season INTEGER,
        round INTEGER,
        race_name TEXT,
        circuit_id TEXT,
        circuit_name TEXT,
        locality TEXT,
        country TEXT,
        date TEXT,
        time TEXT,
        url TEXT
    )
    ''')
    
    # Results table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS results (
        result_id INTEGER PRIMARY KEY AUTOINCREMENT,
        race_id INTEGER,
        driver_id TEXT,
        constructor_id TEXT,
        number INTEGER,
        grid INTEGER,
        position INTEGER,
        position_text TEXT,
        points REAL,
        laps INTEGER,
        time TEXT,
        milliseconds INTEGER,
        fastest_lap INTEGER,
        rank INTEGER,
        fastest_lap_time TEXT,
        fastest_lap_speed TEXT,
        status_id INTEGER,
        status TEXT,
        FOREIGN KEY (race_id) REFERENCES races (race_id),
        FOREIGN KEY (driver_id) REFERENCES drivers (driver_id),
        FOREIGN KEY (constructor_id) REFERENCES constructors (constructor_id)
    )
    ''')
    
    # Qualifying table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS qualifying (
        qualifying_id INTEGER PRIMARY KEY AUTOINCREMENT,
        race_id INTEGER,
        driver_id TEXT,
        constructor_id TEXT,
        number INTEGER,
        position INTEGER,
        q1 TEXT,
        q2 TEXT,
        q3 TEXT,
        FOREIGN KEY (race_id) REFERENCES races (race_id),
        FOREIGN KEY (driver_id) REFERENCES drivers (driver_id),
        FOREIGN KEY (constructor_id) REFERENCES constructors (constructor_id)
    )
    ''')
    
    # Driver standings table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS driver_standings (
        standing_id INTEGER PRIMARY KEY AUTOINCREMENT,
        race_id INTEGER,
        driver_id TEXT,
        constructor_id TEXT,
        position INTEGER,
        position_text TEXT,
        points REAL,
        wins INTEGER,
        FOREIGN KEY (race_id) REFERENCES races (race_id),
        FOREIGN KEY (driver_id) REFERENCES drivers (driver_id),
        FOREIGN KEY (constructor_id) REFERENCES constructors (constructor_id)
    )
    ''')
    
    # Constructor standings table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS constructor_standings (
        standing_id INTEGER PRIMARY KEY AUTOINCREMENT,
        race_id INTEGER,
        constructor_id TEXT,
        position INTEGER,
        position_text TEXT,
        points REAL,
        wins INTEGER,
        FOREIGN KEY (race_id) REFERENCES races (race_id),
        FOREIGN KEY (constructor_id) REFERENCES constructors (constructor_id)
    )
    ''')
    
    conn.commit()
    print("All tables created successfully!")

create_tables()

All tables created successfully!


In [3]:
# Ergast API helper functions
BASE_URL = "http://ergast.com/api/f1/2024"

def fetch_ergast_data(endpoint, limit=1000):
    """Fetch data from Ergast API with pagination"""
    url = f"{BASE_URL}/{endpoint}.json?limit={limit}"
    print(f"Fetching: {url}")
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        time.sleep(0.5)  # Be nice to the API
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def fetch_race_data(round_num, data_type):
    """Fetch specific race data (results, qualifying, etc.)"""
    endpoint = f"{round_num}/{data_type}"
    return fetch_ergast_data(endpoint)

print("API helper functions defined")

API helper functions defined


In [4]:
# Fetch and insert drivers data
def populate_drivers():
    print("Fetching drivers data...")
    data = fetch_ergast_data("drivers")
    
    if data and 'MRData' in data:
        drivers = data['MRData']['DriverTable']['Drivers']
        
        for driver in drivers:
            cursor.execute('''
            INSERT OR REPLACE INTO drivers 
            (driver_id, permanent_number, code, given_name, family_name, date_of_birth, nationality, url)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                driver['driverId'],
                driver.get('permanentNumber'),
                driver.get('code'),
                driver['givenName'],
                driver['familyName'],
                driver['dateOfBirth'],
                driver['nationality'],
                driver['url']
            ))
        
        conn.commit()
        print(f"Inserted {len(drivers)} drivers")
    else:
        print("Failed to fetch drivers data")

populate_drivers()

Fetching drivers data...
Fetching: http://ergast.com/api/f1/2024/drivers.json?limit=1000
Inserted 24 drivers


In [5]:
# Fetch and insert constructors data
def populate_constructors():
    print("Fetching constructors data...")
    data = fetch_ergast_data("constructors")
    
    if data and 'MRData' in data:
        constructors = data['MRData']['ConstructorTable']['Constructors']
        
        for constructor in constructors:
            cursor.execute('''
            INSERT OR REPLACE INTO constructors 
            (constructor_id, name, nationality, url)
            VALUES (?, ?, ?, ?)
            ''', (
                constructor['constructorId'],
                constructor['name'],
                constructor['nationality'],
                constructor['url']
            ))
        
        conn.commit()
        print(f"Inserted {len(constructors)} constructors")
    else:
        print("Failed to fetch constructors data")

populate_constructors()

Fetching constructors data...
Fetching: http://ergast.com/api/f1/2024/constructors.json?limit=1000
Inserted 10 constructors


In [7]:
# Fetch and insert races data
def populate_races():
    print("Fetching races data...")
    # Use the correct endpoint for races
    data = fetch_ergast_data("races")
    
    if data and 'MRData' in data:
        races = data['MRData']['RaceTable']['Races']
        
        for race in races:
            circuit = race['Circuit']
            cursor.execute('''
            INSERT OR REPLACE INTO races 
            (race_id, season, round, race_name, circuit_id, circuit_name, locality, country, date, time, url)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                int(race['round']),
                int(race['season']),
                int(race['round']),
                race['raceName'],
                circuit['circuitId'],
                circuit['circuitName'],
                circuit['Location']['locality'],
                circuit['Location']['country'],
                race['date'],
                race.get('time', ''),
                race['url']
            ))
        
        conn.commit()
        print(f"Inserted {len(races)} races")
        return len(races)
    else:
        print("Failed to fetch races data")
        return 0

total_races = populate_races()

Fetching races data...
Fetching: http://ergast.com/api/f1/2024/races.json?limit=1000
Inserted 24 races


In [8]:
# Fetch and insert results data for all races
def populate_results():
    print("Fetching results data for all races...")
    
    for round_num in range(1, total_races + 1):
        print(f"Fetching results for round {round_num}...")
        data = fetch_race_data(round_num, "results")
        
        if data and 'MRData' in data and data['MRData']['RaceTable']['Races']:
            race = data['MRData']['RaceTable']['Races'][0]
            results = race.get('Results', [])
            
            for result in results:
                fastest_lap = result.get('FastestLap', {})
                cursor.execute('''
                INSERT OR REPLACE INTO results 
                (race_id, driver_id, constructor_id, number, grid, position, position_text, 
                 points, laps, time, milliseconds, fastest_lap, rank, fastest_lap_time, 
                 fastest_lap_speed, status_id, status)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    round_num,
                    result['Driver']['driverId'],
                    result['Constructor']['constructorId'],
                    int(result['number']),
                    int(result['grid']),
                    int(result['position']) if result['position'].isdigit() else None,
                    result['positionText'],
                    float(result['points']),
                    int(result['laps']),
                    result.get('Time', {}).get('time'),
                    result.get('Time', {}).get('millis'),
                    fastest_lap.get('lap'),
                    fastest_lap.get('rank'),
                    fastest_lap.get('Time', {}).get('time'),
                    fastest_lap.get('AverageSpeed', {}).get('speed'),
                    result['status'],
                    result['status']
                ))
            
            print(f"Inserted {len(results)} results for round {round_num}")
        else:
            print(f"No results found for round {round_num}")
    
    conn.commit()
    print("All results data inserted")

populate_results()

Fetching results data for all races...
Fetching results for round 1...
Fetching: http://ergast.com/api/f1/2024/1/results.json?limit=1000
Inserted 20 results for round 1
Fetching results for round 2...
Fetching: http://ergast.com/api/f1/2024/2/results.json?limit=1000
Inserted 20 results for round 2
Fetching results for round 3...
Fetching: http://ergast.com/api/f1/2024/3/results.json?limit=1000
Inserted 19 results for round 3
Fetching results for round 4...
Fetching: http://ergast.com/api/f1/2024/4/results.json?limit=1000
Inserted 20 results for round 4
Fetching results for round 5...
Fetching: http://ergast.com/api/f1/2024/5/results.json?limit=1000
Inserted 20 results for round 5
Fetching results for round 6...
Fetching: http://ergast.com/api/f1/2024/6/results.json?limit=1000
Inserted 20 results for round 6
Fetching results for round 7...
Fetching: http://ergast.com/api/f1/2024/7/results.json?limit=1000
Inserted 20 results for round 7
Fetching results for round 8...
Fetching: http://er

In [9]:
# Fetch and insert qualifying data
def populate_qualifying():
    print("Fetching qualifying data for all races...")
    
    for round_num in range(1, total_races + 1):
        print(f"Fetching qualifying for round {round_num}...")
        data = fetch_race_data(round_num, "qualifying")
        
        if data and 'MRData' in data and data['MRData']['RaceTable']['Races']:
            race = data['MRData']['RaceTable']['Races'][0]
            qualifying_results = race.get('QualifyingResults', [])
            
            for result in qualifying_results:
                cursor.execute('''
                INSERT OR REPLACE INTO qualifying 
                (race_id, driver_id, constructor_id, number, position, q1, q2, q3)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    round_num,
                    result['Driver']['driverId'],
                    result['Constructor']['constructorId'],
                    int(result['number']),
                    int(result['position']),
                    result.get('Q1'),
                    result.get('Q2'),
                    result.get('Q3')
                ))
            
            print(f"Inserted {len(qualifying_results)} qualifying results for round {round_num}")
        else:
            print(f"No qualifying found for round {round_num}")
    
    conn.commit()
    print("All qualifying data inserted")

populate_qualifying()

Fetching qualifying data for all races...
Fetching qualifying for round 1...
Fetching: http://ergast.com/api/f1/2024/1/qualifying.json?limit=1000
Inserted 20 qualifying results for round 1
Fetching qualifying for round 2...
Fetching: http://ergast.com/api/f1/2024/2/qualifying.json?limit=1000
Inserted 20 qualifying results for round 2
Fetching qualifying for round 3...
Fetching: http://ergast.com/api/f1/2024/3/qualifying.json?limit=1000
Inserted 19 qualifying results for round 3
Fetching qualifying for round 4...
Fetching: http://ergast.com/api/f1/2024/4/qualifying.json?limit=1000
Inserted 20 qualifying results for round 4
Fetching qualifying for round 5...
Fetching: http://ergast.com/api/f1/2024/5/qualifying.json?limit=1000
Inserted 20 qualifying results for round 5
Fetching qualifying for round 6...
Fetching: http://ergast.com/api/f1/2024/6/qualifying.json?limit=1000
Inserted 20 qualifying results for round 6
Fetching qualifying for round 7...
Fetching: http://ergast.com/api/f1/2024/7

In [10]:
# Fetch final standings
def populate_standings():
    print("Fetching final driver standings...")
    data = fetch_ergast_data("driverStandings")
    
    if data and 'MRData' in data:
        standings_lists = data['MRData']['StandingsTable']['StandingsLists']
        if standings_lists:
            driver_standings = standings_lists[0]['DriverStandings']
            
            for standing in driver_standings:
                cursor.execute('''
                INSERT OR REPLACE INTO driver_standings 
                (race_id, driver_id, constructor_id, position, position_text, points, wins)
                VALUES (?, ?, ?, ?, ?, ?, ?)
                ''', (
                    total_races,  # Final standings
                    standing['Driver']['driverId'],
                    standing['Constructors'][0]['constructorId'],
                    int(standing['position']),
                    standing['positionText'],
                    float(standing['points']),
                    int(standing['wins'])
                ))
            
            print(f"Inserted {len(driver_standings)} driver standings")
    
    print("Fetching final constructor standings...")
    data = fetch_ergast_data("constructorStandings")
    
    if data and 'MRData' in data:
        standings_lists = data['MRData']['StandingsTable']['StandingsLists']
        if standings_lists:
            constructor_standings = standings_lists[0]['ConstructorStandings']
            
            for standing in constructor_standings:
                cursor.execute('''
                INSERT OR REPLACE INTO constructor_standings 
                (race_id, constructor_id, position, position_text, points, wins)
                VALUES (?, ?, ?, ?, ?, ?)
                ''', (
                    total_races,  # Final standings
                    standing['Constructor']['constructorId'],
                    int(standing['position']),
                    standing['positionText'],
                    float(standing['points']),
                    int(standing['wins'])
                ))
            
            print(f"Inserted {len(constructor_standings)} constructor standings")
    
    conn.commit()
    print("All standings data inserted")

populate_standings()

Fetching final driver standings...
Fetching: http://ergast.com/api/f1/2024/driverStandings.json?limit=1000
Inserted 24 driver standings
Fetching final constructor standings...
Fetching: http://ergast.com/api/f1/2024/constructorStandings.json?limit=1000
Inserted 10 constructor standings
All standings data inserted


In [11]:
# Verify database contents
def verify_database():
    tables = ['drivers', 'constructors', 'races', 'results', 'qualifying', 'driver_standings', 'constructor_standings']
    
    print("\n=== DATABASE VERIFICATION ===")
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        print(f"{table}: {count} records")
    
    # Show sample data
    print("\n=== SAMPLE DATA ===")
    
    print("\nTop 5 Drivers:")
    cursor.execute("SELECT given_name, family_name, nationality FROM drivers LIMIT 5")
    for row in cursor.fetchall():
        print(f"  {row[0]} {row[1]} ({row[2]})")
    
    print("\nFirst 3 Races:")
    cursor.execute("SELECT round, race_name, country, date FROM races ORDER BY round LIMIT 3")
    for row in cursor.fetchall():
        print(f"  Round {row[0]}: {row[1]} in {row[2]} on {row[3]}")
    
    print("\nTop 5 in Championship:")
    cursor.execute("""
        SELECT d.given_name, d.family_name, ds.points, ds.wins 
        FROM driver_standings ds 
        JOIN drivers d ON ds.driver_id = d.driver_id 
        ORDER BY ds.position LIMIT 5
    """)
    for row in cursor.fetchall():
        print(f"  {row[0]} {row[1]}: {row[2]} points, {row[3]} wins")

verify_database()

# Close connection
conn.close()
print("\nDatabase setup complete! File saved as:", DB_PATH)


=== DATABASE VERIFICATION ===
drivers: 24 records
constructors: 10 records
races: 24 records
results: 479 records
qualifying: 479 records
driver_standings: 24 records
constructor_standings: 10 records

=== SAMPLE DATA ===

Top 5 Drivers:
  Alexander Albon (Thai)
  Fernando Alonso (Spanish)
  Oliver Bearman (British)
  Valtteri Bottas (Finnish)
  Franco Colapinto (Argentinian )

First 3 Races:
  Round 1: Bahrain Grand Prix in Bahrain on 2024-03-02
  Round 2: Saudi Arabian Grand Prix in Saudi Arabia on 2024-03-09
  Round 3: Australian Grand Prix in Australia on 2024-03-24

Top 5 in Championship:
  Max Verstappen: 437.0 points, 9 wins
  Lando Norris: 374.0 points, 4 wins
  Charles Leclerc: 356.0 points, 3 wins
  Oscar Piastri: 292.0 points, 2 wins
  Carlos Sainz: 290.0 points, 2 wins

Database setup complete! File saved as: ../data/f1_2024.db
